# Домашние задание

1. Добавить Bias и посчитать для них градиенты.
2. Сравнить градинеты с тем, как считает PyTorch AutoGrad.

Решение 1: Добавить столбец единиц к матрице признаков и получить градиенты по последним индексам в весах. Код для вычисления градиентов не меняется абсолютно, только препроцессинг данных.

Решение 2: Посчитать все руками. Это решение приведено ниже.

In [1]:
import numpy as np
import torch

In [2]:
dtype = torch.float
HARDWARE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = torch.device(HARDWARE)

In [3]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [4]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=DEVICE, dtype=dtype)
y = torch.randn(batch_size, output_size, device=DEVICE, dtype=dtype)

# Randomly initialized weights
w1 = torch.randn(input_size, hidden_size, device=DEVICE, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=DEVICE, dtype=dtype, requires_grad=True)

# Randomly initialized biases
b1 = torch.randn(hidden_size, device=DEVICE, dtype=dtype, requires_grad=True)
b2 = torch.randn(output_size, device=DEVICE, dtype=dtype, requires_grad=True)

# Check that it works
hidden_1 = torch.mm(x, w1) + b1
hidden_relu = torch.clamp_min(hidden_1, 0)
output = torch.mm(hidden_relu, w2) + b2

# Compute loss
loss = (output - y).pow(2).sum().item()
print(loss)

105.85748291015625


In [5]:
### Gradient flow
s = 2 * (output - y)
grad_w2 = torch.mm(s.T, hidden_relu)
grad_b2 = s.sum()
grad_relu = torch.mm(s, w2.T)
grad_relu[hidden_relu<=0] = 0
grad_w1 = torch.mm(x.T, grad_relu)
grad_b1 = grad_relu.sum(dim=0)

In [9]:
learning_rate = 1e-6
for t in range(500):

    ### Forward pass 
    hidden_1 = torch.mm(x, w1) + b1
    hidden_relu = torch.clamp_min(hidden_1, 0)
    output = torch.mm(hidden_relu, w2) + b2

    loss = (output - y).pow(2).sum()
    loss.backward()
    # if t % 100 == 99:
    #     print(t, loss.item())
    
    ### Check autograd values:
    auto_w1 = torch.norm(w1.grad)
    auto_w2 = torch.norm(w2.grad)
    auto_b1 = torch.norm(b1.grad)
    auto_b2 = torch.norm(b2.grad)

    ### Backward pass
    s = 2 * (output - y)
    grad_w2 = torch.mm(hidden_relu.T, s)
    grad_b2 = s.sum()
    grad_relu = torch.mm(s, w2.T)
    grad_relu[hidden_relu<=0] = 0
    grad_w1 = torch.mm(x.T, grad_relu)
    grad_b1 = grad_relu.sum(dim=0)

    ### Check my grad values:
    my_w2 = torch.norm(grad_w2)
    my_w1 = torch.norm(grad_w1)
    my_b1 = torch.norm(grad_b1)
    my_b2 = torch.norm(grad_b2)
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * grad_w1
        w2 -= learning_rate * grad_w2
        b1 -= learning_rate * grad_b1
        b2 -= learning_rate * grad_b2

        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()

    if t % 100 == 99:
        print(f'Iter: {t}, Loss: {loss.item()}')
        print(f'W1: {my_w1}:{auto_w1}, W2: {my_w2}:{auto_w2}')
        print(f'b1: {my_b1}:{auto_b1}, W2: {my_b2}:{auto_b2}')

Iter: 99, Loss: 94.03270721435547
W1: 46.97631072998047:46.97631072998047, W2: 103.0:102.5866470336914
b1: 11.786527633666992:11.786527633666992, W2: 1.3186211585998535:1.3186211585998535
Iter: 199, Loss: 92.79016876220703
W1: 45.59490203857422:45.59490203857422, W2: 99.0:98.95589447021484
b1: 10.796223640441895:10.796223640441895, W2: 0.05169105529785156:0.05169105529785156
Iter: 299, Loss: 91.63225555419922
W1: 44.28825759887695:44.28825759887695, W2: 95.0:95.48926544189453
b1: 9.871001243591309:9.871001243591309, W2: 1.1476178169250488:1.1476178169250488
Iter: 399, Loss: 90.5518798828125
W1: 43.05105209350586:43.05105209350586, W2: 92.0:92.17737579345703
b1: 9.006209373474121:9.006209373474121, W2: 2.2829675674438477:2.2829675674438477
Iter: 499, Loss: 89.5426025390625
W1: 41.878379821777344:41.878379821777344, W2: 89.0:89.01138305664062
b1: 8.197602272033691:8.197602272033691, W2: 3.3577866554260254:3.3577866554260254


Все отлично совпадает с вычисленными автоматически градиентами!